In [3]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained model and tokenizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model and tokenizer from the saved folder
model = GPT2LMHeadModel.from_pretrained('./finetuned_model')
tokenizer = GPT2Tokenizer.from_pretrained('./finetuned_model')

# Ensure pad_token_id is set
tokenizer.pad_token = tokenizer.eos_token

# Define the function to generate the response
def generate_response(user_input):
    # Tokenize the input
    input_ids = tokenizer.encode(user_input, return_tensors='pt')
    
    # Create attention mask: 1 for real tokens, 0 for padding tokens
    attention_mask = (input_ids != tokenizer.pad_token_id).to(torch.long)
    
    # Generate the response with sampling parameters
    output = model.generate(
    input_ids, 
    attention_mask=attention_mask, 
    max_length=100, 
    num_return_sequences=1, 
    no_repeat_ngram_size=2,
    top_k=10,              # Lower top_k to reduce randomness
    top_p=0.8,             # Lower top_p for more controlled output
    temperature=0.5,       # Lower temperature for less randomness
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True
)
    
    # Decode and return the generated response
    return tokenizer.decode(output[0], skip_special_tokens=True)
    


In [4]:
import random
import json
import random
import re


In [5]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import random

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')  # Replace with your fine-tuned model path
tokenizer = GPT2Tokenizer.from_pretrained('./finetuned_model') 


tokenizer.pad_token = tokenizer.eos_token 


# Define the intent keywords and responses
responses = {
    "greeting": [
        "Hello! How can I assist you today?",
        "Hi there! How can I help you?",
        "Greetings! How can I support you?"
    ],
    "about_calisnova": [
        "Calisnova is an innovative platform focused on empowering students with essential life skills.",
        "At Calisnova, we believe in fostering personal growth through practical, real-world learning experiences."
    ],
    "courses": [
        "We offer a variety of courses to help students develop skills in leadership, innovation, and critical thinking.",
        "Our courses are designed to equip students with the skills they need to succeed in the modern world."
    ],
    "contact": [
        "You can reach us via email at contact@calisnova.com.",
        "Feel free to contact us at support@calisnova.com for more information."
    ],
    "events": [
        "We regularly host workshops and seminars on various topics. Check our events calendar for the latest updates.",
        "Keep an eye on our events page for upcoming workshops and seminars."
    ],
    "leadership_skills": [
        "Leadership skills are critical for personal and professional success. We offer training in areas like decision-making and team management.",
        "Our leadership training programs focus on building confidence, team building, and self-leadership."
    ],
    "innovation_skills": [
        "Innovation is the key to progress. We help students develop creativity, problem-solving, and critical thinking skills.",
        "Through our innovation programs, we nurture students' creative potential and foster new ideas."
    ],
    "critical_thinking": [
        "Critical thinking is essential for making informed decisions. We teach skills like logical reasoning and problem-solving.",
        "Our courses help students develop the ability to analyze situations and think critically."
    ],
    "programs_overview": [
        "We offer programs in leadership, innovation, critical thinking, and more. Check out our offerings for more details.",
        "Our programs are designed to provide hands-on learning and practical experience in various fields."
    ],
    "social_media_awareness": [
        "We teach students how to use social media responsibly and maintain their digital well-being.",
        "Our social media awareness programs help students navigate online spaces safely and effectively."
    ],
    "unknown": [
        "Sorry, I didn't quite understand that. Could you please rephrase?",
        "I’m not sure how to help with that. Can you clarify?"
    ]
}

intent_keywords = {
    "greeting": ["hello", "hi", "hey", "greetings", "howdy", "morning", "evening"],
    "about_calisnova": ["calisnova", "about", "what is", "who are we", "tell me about", "mission"],
    "courses": ["courses", "classes", "offer", "teach", "programs", "subjects"],
    "contact": ["contact", "email", "reach us", "how to contact"],
    "events": ["events", "workshops", "seminars", "upcoming events", "calendar"],
    "leadership_skills": [
        "leadership", "team building", "motivation", "public speaking", "conflict resolution",
        "decision making", "lead teams", "confidence building", "self-leadership", "responsibility"
    ],
    "innovation_skills": [
        "innovation", "creativity", "design thinking", "problem-solving", "hackathon",
        "idea generation", "critical thinking", "experimentation", "new ideas", "creative thinking"
    ],
    "critical_thinking": [
        "critical thinking", "analysis", "logical reasoning", "decision making", "problem solving",
        "assumptions", "evaluations", "debate", "analyze", "logical conclusions"
    ],
    "programs_overview": [
        "programs", "courses", "training", "workshops", "sessions", "personal development",
        "skill development", "learning", "overview of services", "what do you teach"
    ],
    "social_media_awareness": ["social media", "social media awareness", "online safety", "digital well-being", "social media etiquette"]
}



# Function to get intent based on user input
def get_intent(user_input):
    user_input = user_input.lower()

    # Match greeting-related intents
    for keyword in intent_keywords["greeting"]:
        if keyword in user_input:
            return "greeting"
    
    # Match about_calisnova-related intents
    for keyword in intent_keywords["about_calisnova"]:
        if keyword in user_input:
            return "about_calisnova"
    
    # Match courses-related intents
    for keyword in intent_keywords["courses"]:
        if keyword in user_input:
            return "courses"
    
    # Match contact-related intents
    for keyword in intent_keywords["contact"]:
        if keyword in user_input:
            return "contact"
    
    # Match events-related intents
    for keyword in intent_keywords["events"]:
        if keyword in user_input:
            return "events"
    
    # Match leadership_skills-related intents
    for keyword in intent_keywords["leadership_skills"]:
        if keyword in user_input:
            return "leadership_skills"
    
    # Match innovation_skills-related intents
    for keyword in intent_keywords["innovation_skills"]:
        if keyword in user_input:
            return "innovation_skills"
    
    # Match critical_thinking-related intents
    for keyword in intent_keywords["critical_thinking"]:
        if keyword in user_input:
            return "critical_thinking"
    
    # Match programs_overview-related intents
    for keyword in intent_keywords["programs_overview"]:
        if keyword in user_input:
            return "programs_overview"
    
    # Match social_media_awareness-related intents
    for keyword in intent_keywords["social_media_awareness"]:
        if keyword in user_input:
            return "social_media_awareness"
    
    # Default case if no intent matches
    return "unknown"




def generate_finetuned_response(user_input):
    inputs = tokenizer(user_input, return_tensors="pt", padding=True, truncation=True, max_length=512)

    outputs = model.generate(
    inputs["input_ids"], 
    max_length=150, 
    num_return_sequences=1, 
    pad_token_id=tokenizer.eos_token_id, 
    attention_mask=inputs["attention_mask"],
    no_repeat_ngram_size=2  # This prevents repeated n-grams
)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response  # Ensure this is only returned once


def get_response(intent):
    if intent == "unknown":
        return "I am not sure how to respond to that. Can you ask something else?"
    return random.choice(responses.get(intent, responses["unknown"]))

def chatbot_response(user_input):
    intent = get_intent(user_input)
    
    # If a predefined intent is matched, return the predefined response
    if intent != "unknown":
        return get_response(intent)
    
    # Otherwise, use the fine-tuned model to generate a dynamic response
    return generate_finetuned_response(user_input)

# Chat loop
def chat():
    print("Chatbot: Hello! I am Calisnova's assistant. How can I help you today?")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break
        response = chatbot_response(user_input)
        print(f"Chatbot: {response}")

# Start the chatbot
chat()


Chatbot: Hello! I am Calisnova's assistant. How can I help you today?


RuntimeError: cannot reshape tensor of 0 elements into shape [-1, 0] because the unspecified dimension size -1 can be any value and is ambiguous